In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
end20=pd.read_csv("/kaggle/input/2020-democratic-primary-endorsements/endorsements-2020.csv")

In [ ]:
end20.head()

If we want to predict New Hampshire based on endorsements, it seems like proximity in time and significance score are important. Let's take a time series approach and find the SMA smoothed series of endorsements per candidate, starting with NH endorsers and then looking at surrounding states.
First get a time series of candidate points per date, by changing the format of this data to use date as an index and endorsee as column, with the values as the total endorsement score for that day

First, fix date data type. We can get rid of the first one in 2017 since it was for John Delaney and afterwards we jump to 2019...

In [ ]:
end20['date']=pd.to_datetime(end20['date'])

In [ ]:
end20=end20.iloc[1:]

In [ ]:
start=min(end20.date)
end=max(end20.date)

In [ ]:
idx=pd.date_range(start,end,freq='D')
L=len(idx)
names=list(end20.endorsee.unique())

In [ ]:
#drop nan
names=names[:-1]

In [ ]:
def blank():
    outdf=pd.DataFrame(index=idx,data={names[i]:L*[0] for i in range(len(names))})
    return outdf
def endts(df):
    outdf=blank()
    #remove NaT
    df=df.dropna(subset=['date'])
    for i in range(len(df)):
        date=df.iloc[i].date
        ende=df.iloc[i].endorsee
        pnts=df.iloc[i].points
        outdf.loc[date,ende]+=pnts
    return outdf

In [ ]:
allts=endts(end20)

In [ ]:
allts.head()

In [ ]:
def rollavgs(df,ende,pds):
    return sum([df[ende].rolling(window=i).mean() for i in pds])/len(pds)

Now let us look at the 7-day rolling avgs for the remaining candidates (Biden, Sanders, Klobuchar, Warren, Buttigieg, Bloomberg, Steyer)

In [ ]:
import matplotlib.pyplot as plt
plt.close()
for name in names:
    rollavgs(allts,name,[7]).plot()
plt.show()

Biden is clearly winning the endorsement race...let us limit this analysis to NH now.

In [ ]:
nh=end20.loc[end20['state']=='NH',:]

In [ ]:
nhts=endts(nh)

In [ ]:
plt.close()
for name in names:
    rollavgs(nhts,name,[7]).plot()
plt.legend(loc='upper left')
plt.show()

This doesn't seem to be working. Let's look at individual candidates to start instead.

In [ ]:
biden=end20.loc[end20['endorsee']=='Joe Biden',:]

In [ ]:
biden.groupby('endorser party').count()

In [ ]:
def byparty(cand):
    df=end20.loc[end20['endorsee']==cand,:]
    return df.groupby('endorser party').count()

We know Biden and Buttigieg lead in Iowa. Let's look at Iowa alone for their endorsements

In [ ]:
ia=end20.loc[end20['state']=='IA',:]
iats=endts(ia)

In [ ]:
plt.close()
for name in ['Joe Biden','Bernie Sanders','Amy Klobuchar','Elizabeth Warren','Pete Buttigieg','Michael Bloomberg']:
    rollavgs(iats,name,[7]).plot()
plt.legend(loc='upper left')
plt.show()

In [ ]:
plt.close()
for name in ['Joe Biden','Bernie Sanders','Amy Klobuchar','Elizabeth Warren','Pete Buttigieg','Michael Bloomberg']:
    rollavgs(nhts,name,[7]).plot()
plt.legend(loc='upper left')
plt.show()

In [ ]:
ia.groupby('endorsee').sum()['points']

In [ ]:
ia.groupby('endorsee').count()['endorser']

In [ ]:
ia.groupby('endorsee').mean()['points']

In [ ]:
nh.groupby('endorsee').mean()['points']

In [ ]:
sc=end20.loc[end20['state']=='SC',:]

In [ ]:
sc.groupby('endorsee').mean()['points']

In [ ]:
end20.dropna(subset=['date']).groupby('state').count()['points'].sort_values(ascending=False)

In [ ]:
tx=end20.loc[end20['state']=='TX',:]
tx.groupby('endorsee').mean()['points']

In [ ]:
texavgend=tx.groupby('endorsee').mean()['points']

In [ ]:
texavgend.index